In [141]:
#%###################################
#%#           Parity trees          #
#%###################################
#%ParityTree(num_of_sites,Parents)
struct PTree
    M::Int64
    Parents::Array{Int64}
end

#-1 indicates the node has no parent
function JWTree(M)
    Parents=-ones(M,1)
    return PTree(M,Parents)
end


#%Parity tree functions
#%--
#%function Ancestors(      j,PT::PTree)
#%function Children(       j,PT::PTree)
#%function YoungerCousins( j,PT::PTree)
#%function DisjointRoots(  j,PT::PTree)
#%function Progeny(        j,PT::PTree)

LoadError: syntax: extra token "Parents" after end of expression

In [101]:
function Ancestors(j,PT::PTree,verbose::Bool=false)
    A=[];
    cur=j;
    ctr=1;
    while ctr<1000
        if(verbose)
            println("ctr - ",ctr)
        end
        if(PT.Parents[cur]==-1)
            if(verbose)
                println("breaking val of cur=",cur)
            end
            break;
        else
            A=[A; PT.Parents[cur]];
            cur=PT.Parents[cur];
        end
        
        ctr=ctr+1
    
    end
    if ctr>PT.M
        if(verbose)
            println("ctr - ",ctr,"; PT.M - ",PT.M)
        end
        print("Warning, the loop didn't close automatically")
    end
    
    return A;
end



Ancestors (generic function with 2 methods)

In [104]:
Ancestors(1,pt,true)

ctr - 1
ctr - 2
ctr - 3
breaking val of cur=3


2-element Array{Any,1}:
 2
 3

In [74]:
A=[];
j=1;
cur=j;
ctr=1;

In [106]:
jw=JWTree(3)
Ancestors(1,jw)

0-element Array{Any,1}

In [76]:
function ParityTree(N)

  paritytree=-ones(N,1);

  for i=1:(N-1)
    paritytree[i]=i+1;
  end

  return PTree(N,paritytree);
end


ParityTree (generic function with 1 method)

In [155]:
pt=ParityTree(3)
Ancestors(1,pt)

2-element Array{Any,1}:
 2
 3

In [90]:

function Children(j,PT::PTree)
    C=[];
    for k=1:PT.M
        if(PT.Parents[k]==j)
            C=[C; k];
        end
    end
    return C;
end



Children (generic function with 1 method)

In [110]:
Children(3,pt)

1-element Array{Any,1}:
 2

In [94]:
function Progeny(j,tree::PTree)
  #children's children
  all_kids=[];

  ctr=0;
  g0kids=Children(j,tree);


  while(length(g0kids)>0)
    #Basis cases to illustrate the logic
    #
    # kids=Children(j)
    # grandkids=[];
    # for i=1:length(kids)
    #   child=kids[i];
    #   grandkids=[grandkids Children(child)];
    # end
    #
    # greatgrandkids=[];
    # for i=1:length(grandkids)
    #   grandchild=grandkids[i];
    #   greatgrandkids=[greatgrandkids Children(grandchild)];
    # end

    #g0 is the previous generation, g1 is the next generation
    g1kids=[];
    for i=1:length(g0kids)

      #pull child
      child0=g0kids[i];

      g1kids=append!(g1kids,Children(child0,tree));
    end

    #record this generation
    all_kids=append!(all_kids,g0kids);

    #check the next generation
    g0kids=g1kids;

    ctr=ctr+1;
    #prevent excessive nesting
    if(ctr>100)
      print("\n\tWarning: Excessive nesting, pruning\n")
      return;
    end
  end

  return all_kids;
end



Progeny (generic function with 1 method)

In [111]:
Progeny(3,pt)

2-element Array{Any,1}:
 2
 1

In [97]:

function YoungerCousins(j,PT::PTree)
    K=[];
    A=Ancestors(j,PT)
    for k=1:length(A)
        Ck=Children(A[k],PT);
        for kk=1:length(Ck)
            if(Ck[kk]<j)
                K=[K; Ck[kk]]
            end
        end
    end
    return K;
end



YoungerCousins (generic function with 1 method)

In [112]:
YoungerCousins(3,jw)

0-element Array{Any,1}

In [113]:

function DisjointRoots(j,PT::PTree)
    L=[];

    for k=j-1:-1:1
        if PT.Parents[k]==-1;
            L=[L; k];
        end
    end

    return L
end



DisjointRoots (generic function with 1 method)

In [118]:
DisjointRoots(3,jw)

2-element Array{Any,1}:
 2
 1

In [116]:
DisjointRoots(2,pt)

0-element Array{Any,1}

In [154]:
function ShiftTreeIndices(shift,parent_tree)
  shifted_tree=parent_tree.+shift;

  #if it used to be empty it needs to reset
  shifted_tree[shifted_tree .== shift-1].=-1;
  return shifted_tree
end



ShiftTreeIndices (generic function with 1 method)

In [158]:
ShiftTreeIndices(4,pt.Parents)

3×1 Array{Int64,2}:
  6
  7
 -1

In [156]:
st=pt.Parents.+1
idx=st.==0
st[3]=-1
#st[idx]=-1

-1

In [157]:
st[st.==-1].=0

1-element view(::Array{Int64,1}, [3]) with eltype Int64:
 0

In [153]:
st

3×1 Array{Int64,2}:
 3
 4
 0

In [159]:
pt

PTree(3, [2; 3; -1])

In [160]:
function CombineTrees(treeL,treeR)
  m=length(treeL)
  shifted_treeR=ShiftTreeIndices(m,treeR)
  return [treeL;shifted_treeR]
end



CombineTrees (generic function with 1 method)

In [163]:
ct=CombineTrees(pt.Parents,pt.Parents)

6×1 Array{Int64,2}:
  2
  3
 -1
  5
  6
 -1

In [183]:
ct==[2;3;-1;5;6;-1]

false

In [194]:
Array(ct)

6×1 Array{Int64,2}:
  2
  3
 -1
  5
  6
 -1

In [200]:
Array([2;3;-1;5;6;-1])≈ Array(ct)

true

In [196]:
ct[2]

3

In [197]:
ct

6×1 Array{Int64,2}:
  2
  3
 -1
  5
  6
 -1

In [169]:
ct[3]

-1

In [170]:
ct[4]

5

In [204]:
function BetaMatrix(PT::PTree)

  #make MxM zeros matrix
  B=zeros(Int64,PT.M,PT.M)

  #for each node in tree
  for j=1:PT.M

    #get progeny list
    pj=Progeny(j,PT)

    #mark appropriate matrix elements

    #use the state ordering from eq (24) of Seeley12

    #equation 24 of peter's paper has a strange ordering
    #based on b_i=\sum B_{ij} f_j instead of the usual change of basis formula
    # b_i=\sum B_{ji} f_j

    B[j,j]=1;
    for k=1:length(pj)
      B[PT.M+1-j,PT.M+1-pj[k]]=1;
    end

  end

  #return matrix
  return B;
end

BetaMatrix (generic function with 2 methods)

In [206]:
BetaMatrix(jw)

3×3 Array{Int64,2}:
 1  0  0
 0  1  0
 0  0  1

In [207]:

#%######################################
#% Fenwick tree and related structures #
#%######################################
#%--
#% tree = MakeFenwickTree(M)
#% PT = JWTree(M)
#% PT = BKTree(M)
#% PT = SBKTree(M,stride_length)
#% PT = ParityTree(M)

function MakeFenwickTree(N)

    if(N==0)
      return [];
    end
    #global variables with respect to FENWICK
    tree=-ones(N,1)
    ctr=0;

    function FENWICK(L,R)
        #The recursive algorithm Vojta suggested in our paper


        if(ctr>100)
            #prevent excessive nesting
            print("\n\tWarning: Excessive nesting, pruning\n")
            return;
        end

        #algorithm is to connect from middle between L & R to right then recuse
        #on each half. Be careful about floor versus ceiling.

        if(L==R)            #if L=R then there is no middle, so we're done
            return;
        end
        n=Int64(floor((L+R)/2))

        #Connect middle to R
        #+1 because we count from 1 in this language (Julia)
        tree[n+1]=R+1;

        #branch
        ctr=ctr+1;
        FENWICK(L+0.0, n)
        FENWICK(1.0+n, R)

    end

    #run the recursive algorithm on (0,N-1) and it generates the BK tree over N
    #nodes.
    FENWICK(0.0,N-1.0)
    return tree
end

function JWTree(M)
    Parents=-ones(M,1)
    return PTree(M,Parents)
end

function BKTree(M)
  return PTree(M,MakeFenwickTree(M))
end

function SBKTree(M,L)
  #L=stride_length;

  #how many strides?
  n=M/L;
  n=n-n%1;
  #remainder
  r=M-n*L;

  sktree=[];
  #Now we just need to make n strides
  for i=1:n
    sktree=CombineTrees(MakeFenwickTree(L),sktree);
  end
  #plus the remaining nodes

  sktree=CombineTrees(MakeFenwickTree(r),sktree);

  return PTree(M,sktree);
end

function ParityTree(M)

  paritytree=-ones(M,1);

  for i=1:(M-1)
    paritytree[i]=i+1;
  end

  return PTree(M,paritytree);
end



ParityTree (generic function with 1 method)

In [209]:
bk=BKTree(4)

PTree(4, [2; 4; 4; -1])

In [216]:
#%###################################
#%           Spin algebra           #
#%###################################
#%We'll deal with systems of spin-half particles. These are governed by the
#%Pauli spin matrices.  Additionally, the raising and lowering spin operators are
#%useful.  We'll assign them all numbers and define algebra as a multiplication
#%table
#%
#% Pauli basis: I, X, Y, Z
#% Index basis: P_00, P_01, P_10, P_11
#%   with
#%  P_00 = (I+Z)/2
#%  P_11 = (I-Z)/2
#%  P_01 = S_+ = (X+iY )/2
#%  P_10 = S_- = (X-iY )/2
#%
#% Basic algebraic relations
#%   XY= iZ    YX= -iZ
#%   ZX= iY    XZ= -iY
#%   YZ= iX    ZY= -iX
#%  matrix    =  GetSpinAlgebra()
#%  MatrixID  =  MultiplyPauli(MatrixID, MatrixID)

########### Z BASIS ################
#Pauli Matrix basis
ii=[1. 1.0; 0.0  1.0];
x= [0  1.0; 1.0  0.0];
y= [0 -1im; 1im  0.0];
z= [1. 0.0; 0.0 -1.0];
#Index basis
p00=[1. 0.; 0. 0.];
p01=[0. 1.; 0. 0.];
p10=[0. 0.; 1. 0.];
p11=[0. 0.; 0. 1.];

# We use integer MatrixIDs for coding the algebra
I   =1;
X   =2;
Y   =3;
Z   =4;
P00 =5;
P01 =6;
P10 =7;
P11 =8;

function GetSpinAlgebra()

   SPIN_ALGEBRA=zeros(Complex,8,8)
   #These are the MatrixIDs for coding algebra
   I   =1;
   X   =2;
   Y   =3;
   Z   =4;
   P00 =5;
   P01 =6;
   P10 =7;
   P11 =8;

   #ROW I.j
   #   [ 11  1X  1Y  1Z  1 P_00   1 P_01   1 P_10   1 P_11
   # = [ +1   X   Y   Z  +P_00    +P_01    +P_10    +P_11
   SPIN_ALGEBRA[I,I]=I;
   SPIN_ALGEBRA[I,X]=X;
   SPIN_ALGEBRA[I,Y]=Y;
   SPIN_ALGEBRA[I,Z]=Z;
   SPIN_ALGEBRA[I,P00]=P00;
   SPIN_ALGEBRA[I,P01]=P01;
   SPIN_ALGEBRA[I,P10]=P10;
   SPIN_ALGEBRA[I,P11]=P11;

   #ROW X.j
   #   [ X1  XX  XY  XZ  X P_00   X P_01   X P_10   X P_11]
   # = [ +X  +1 +iZ -iY  +P_10    +P_11    +P_00    +P_01 ]
   SPIN_ALGEBRA[X,I]	=    X;
   SPIN_ALGEBRA[X,X]	=    I;
   SPIN_ALGEBRA[X,Y]	=  1im*Z;
   SPIN_ALGEBRA[X,Z]	= -1im*Y;
   SPIN_ALGEBRA[X,P00]     =   P10;
   SPIN_ALGEBRA[X,P01]     =   P11;
   SPIN_ALGEBRA[X,P10]     =   P00;
   SPIN_ALGEBRA[X,P11]     =   P01;

   #ROW Y.j
   #   [  Y1  YX  YY  YZ   Y P_00   Y P_01   Y P_10   Y P_11 ]
   # = [  +Y -iZ  +1 +iX   +iP_10   +iP_11   -iP_00   -iP_01 ]
   SPIN_ALGEBRA[Y,I]	=    Y;
   SPIN_ALGEBRA[Y,X]	= -1im*Z;
   SPIN_ALGEBRA[Y,Y]	=    I;
   SPIN_ALGEBRA[Y,Z]	= +1im*X;
   SPIN_ALGEBRA[Y,P00]     = +1im*P10;
   SPIN_ALGEBRA[Y,P01]     = +1im*P11;
   SPIN_ALGEBRA[Y,P10]     = -1im*P00;
   SPIN_ALGEBRA[Y,P11]     = -1im*P01;

   #ROW Z.j
   #     Z1  ZX  ZY  ZZ   Z  P_00    Z  P_01    Z  P_10     Z  P_11 ]
   # = [ +Z +iY -iX  +1    +P_00      +P_01      -P_10       -P_11  ]
   SPIN_ALGEBRA[Z,I]	=    Z;
   SPIN_ALGEBRA[Z,X]       = +1im*Y;
   SPIN_ALGEBRA[Z,Y]	= -1im*X;
   SPIN_ALGEBRA[Z,Z]	=    I;
   SPIN_ALGEBRA[Z,P00] 	=   P00;
   SPIN_ALGEBRA[Z,P01]	=   P01;
   SPIN_ALGEBRA[Z,P10]	=  -P10;
   SPIN_ALGEBRA[Z,P11]	=  -P11;

   #ROW P00.j
   #   [ P_00 1   P_00 X   P_00 Y   P_00 Z    P_00 P_00   P_00 P_01   P_00 P_10   P_00 P_11
   # = [  +P_00    +P_01   -iP_01    +P_00      P_00         P_01         0          0
   SPIN_ALGEBRA[P00,I]	=    P00;
   SPIN_ALGEBRA[P00,X]	=    P01;
   SPIN_ALGEBRA[P00,Y]	= -1im*P01;
   SPIN_ALGEBRA[P00,Z]	=    P00;
   SPIN_ALGEBRA[P00,P00]   =    P00;
   SPIN_ALGEBRA[P00,P01]   =    P01;
   SPIN_ALGEBRA[P00,P10]   =     0;
   SPIN_ALGEBRA[P00,P11]   =     0;

   #ROW P01.j
   #   [  P_01 1   P_01 X   P_01 Y   P_01 Z   P_01 P_00   P_01 P_01   P_01 P_10   P_01 P_11
   # = [   +P_01    +P_00   +iP_00    -P_01       0            0         P_00       P_01
   SPIN_ALGEBRA[P01,I]	=    P01;
   SPIN_ALGEBRA[P01,X]	=    P00;
   SPIN_ALGEBRA[P01,Y]	= +1im*P00;
   SPIN_ALGEBRA[P01,Z]	=   -P01;
   SPIN_ALGEBRA[P01,P00]   =     0;
   SPIN_ALGEBRA[P01,P01]   =     0;
   SPIN_ALGEBRA[P01,P10]   =    P00;
   SPIN_ALGEBRA[P01,P11]   =    P01;

   #ROW P10.j
   #   [  P_10 1   P_10 X   P_10 Y   P_10 Z   P_10 P_00   P_10 P_01   P_10 P_10   P_10 P_11
   # = [   +P_10    +P_11   -iP_11    +P_10       P_10         P_11         0          0
   SPIN_ALGEBRA[P10,I]	=    P10;
   SPIN_ALGEBRA[P10,X]	=    P11;
   SPIN_ALGEBRA[P10,Y]	= -1im*P10;
   SPIN_ALGEBRA[P10,Z]	=   +P01;
   SPIN_ALGEBRA[P10,P00]   =    P10;
   SPIN_ALGEBRA[P10,P01]   =    P11;
   SPIN_ALGEBRA[P10,P10]   =     0;
   SPIN_ALGEBRA[P10,P11]   =     0;

   #ROW P11.j
   #   [  P_11 1   P_11 X   P_11 Y   P_11 Z   P_11 P_00   P_11 P_01   P_11 P_10   P_11 P_11
   # = [   +P_11    +P_10   +iP_10    -P_11       0            0         P_10       P_11
   SPIN_ALGEBRA[P11,I]	=    P11;
   SPIN_ALGEBRA[P11,X]	=    P10;
   SPIN_ALGEBRA[P11,Y]	= +1im*P10;
   SPIN_ALGEBRA[P11,Z]	=   -P11;
   SPIN_ALGEBRA[P11,P00]   =     0;
   SPIN_ALGEBRA[P11,P01]   =     0;
   SPIN_ALGEBRA[P11,P10]   =    P10;
   SPIN_ALGEBRA[P11,P11]   =    P11;

   return SPIN_ALGEBRA;
end



GetSpinAlgebra (generic function with 1 method)

In [226]:
function MultiplyPauli(A,B)
  algebra=GetSpinAlgebra();
  return algebra[A,B];
end

MultiplyPauli (generic function with 1 method)

In [227]:
MultiplyPauli(Z,X)

0 + 3im

In [228]:

#%###################################
#%          Tensor algebra          #
#%###################################
#% Spin chain multiplication
#%
#% We need both multiplication and addition of Pauli sums.   We will store the
#% spin chain operators as a list with a complex prefactor. Plain ASCII output
#% is required for the final output of the result.  The present
#% example is for small matrix size so we will explicitly store each tensor
#% contribution.
#%
#% PS    = PauliSum(PL1,PL2,...,PLn)
#% PList = PauliList(Prefactor,OperatorList)
#%
#% Pauli list functions
#%  PList        =  StandardForm(PList)
#%  PList        =  Minus(Plist)
#%  PList        =  MakeLocalOperator(MatrixID, tensor_index, num_spins)
#%  PList        =  MultiplyPauliList(PauliL::PauliList,PauliR::PauliList)
#%  PLists       =  MultiplyPauliLists(PauliSumL,PauliSumR)
#%       PrintPauliList(PauliList)
#%       PrintPauliLists(PauliLists)
#%  PList        =  Sum(PList)

#Pauli list
temp = @isdefined PauliList
if( temp == false ) 
  struct PauliList
    Prefactor::Complex
    List::Array{Complex}
  end
end

function StandardForm(P::PauliList)

  #a little error catching to make sure MatrixIDs are okay
  if( 0!=sum(abs(P.List).>8.0) || 0!=sum(abs(P.List).<1.0))
    println("Warning invalid MatrixIDs in StandardForm function")
  end

  #pull StandardForm
  b=(P.List)./abs(P.List);

  #if entry of list is zero extracting the phase via A/|A| is invalid. Hence,
  ## !isnan ## command. All MatrixIDs should be 1,...,8 so this isn't necessary.
  b=prod( b[!isnan(b)] );

  P.Prefactor=P.Prefactor*b;

  #put everything as standard matrix IDs
  P.List=Complex.(round(abs(P.List)));

  return P;
end


function MakeLocalOperator(MatrixID, j, M)
  list=ones(M);
  list[j]=MatrixID;
  return PauliList(1,list)
end

function MultiplyPauliList(L::PauliList,R::PauliList)
  algebra=GetSpinAlgebra();

  out=zeros(Complex,max(length(L.List),length(R.List)));
  if(length(L.List)!=length(R.List))
    println("Warning: tensor products must be same length. Returning 0")
    return 0;
  end


  F=L.Prefactor*R.Prefactor;
  for k=1:length(L.List)
    out[k]=algebra[Int64(abs(L.List[k])),Int64(abs(R.List[k]))]
  end

  return StandardForm(PauliList(F,out));
end

function MultiplyPauliLists(aL,aR)
  #Here aL and aR are the arrays of PauliLists
  out=Array{PauliList}([]);
  for i=1:length(aL)
    for j=1:length(aR)
      if length(out)==0
        out=[MultiplyPauliList( aL[i], aR[j] )]
      else
        out=[out;MultiplyPauliList(aL[i],aR[j])]
      end
    end
  end
  return out;
end

function PrintPauliList(P::PauliList)
  #correct the state
  P=StandardForm(P)

  #These are the MatrixIDs for coding the algebra
  I   =1;
  X   =2;
  Y   =3;
  Z   =4;
  P00 =5;
  P01 =6;
  P10 =7;
  P11 =8;

  #printing
  print("(")
  if(real(P.Prefactor)>=0) #add plus sign to make it look better
    print("+")
  end
  print(P.Prefactor)
  print(") ")
  for i=1:length(P.List)
    if(P.List[i]==I)
      print("  I  ")
    end
    if(P.List[i]==X)
      print("  X  ")
    end
    if(P.List[i]==Y)
      print("  Y  ")
    end
    if(P.List[i]==Z)
      print("  Z  ")
    end
    if(P.List[i]==P00)
      print(" P00 ")
    end
    if(P.List[i]==P01)
      print(" P01 ")
    end
    if(P.List[i]==P10)
      print(" P10 ")
    end
    if(P.List[i]==P11)
      print(" P11 ")
    end
  end
  println("")
  return
end

function PrintPauliLists(arrayPL)

  if(string(typeof(arrayPL))=="PauliList")
    arrayPL=[arrayPL];
  end

  if(string(typeof(arrayPL[1]))!="PauliList")
    println("Warning: bad input; expected array of PauliLists in function Print_sum")
  end

  for k=1:length(arrayPL)
    if(k>1)
      print("    +")
    else
      print("op = ")
    end
    Print(arrayPL[k])
  end
end


#adds like list terms for simplification (S. Manski 2016)
#input: unsimplified PauliList
#output: PauliList with combined like terms
function Sum(arrayPL)

  arrayA=PauliList[];
  for i=1:length(arrayPL)
    for j=i+1:length(arrayPL)
      if( arrayPL[i].List == arrayPL[j].List && arrayPL[i].Prefactor!=0)
          arrayPL[i].Prefactor=arrayPL[i].Prefactor+arrayPL[j].Prefactor;
          arrayPL[j].Prefactor=0;
      end
    end
    if(arrayPL[i].Prefactor!=0)
      push!(arrayA,arrayPL[i]);
    end
  end
  return arrayA
end




# Wrapper functions
#  ans    =  Multiply(A,B)
#  output =  Print(A)


function Multiply(A,B)

  if typeof(A)<: Number
    #put the scalar second
    temp=A;
    A=B;
    B=temp;
  end


  if  typeof(A) <: Vector{PauliList} && typeof(B) <: Number
    for a in A
      a.Prefactor=a.Prefactor * B;
    end
    return A;
  end

  if typeof(A) <: PauliList && typeof(B) <: Number
    A.Prefactor=A.Prefactor*B;
    return A;
  end


  if(string(typeof(A))== "PauliList" && string(typeof(B))== "PauliList")
		return MultiplyPauliList(A,B)
	end

	if(   (     string(typeof(A))== "Array{PauliList,1}"  ||      string(typeof(A))== "Vector{PauliList}" )
           && (     string(typeof(B))== "Array{PauliList,1}"  ||      string(typeof(B))== "Vector{PauliList}" ))
		return MultiplyPauliLists(A,B)
	end

	println("Types: ",typeof(A)," , ",typeof(B))
	return A*B
end

#TODO OVERLOAD * TO USE Multiply
#import Base.*
#println(*(3,9))

function Print(P)
  if(string(typeof(P))=="PauliList")
    PrintPauliList(P)
    return;
  end
  if( string(typeof(P))=="Array{PauliList,1}")
    PrintPauliLists(P)
    return;
  end
  if( string(typeof(P))=="Vector{PauliList}" )
    PrintPauliLists(P)
    return;
  end
  println(typeof(P))
  println(P)
  return;
end




Print (generic function with 1 method)